# Dependencies

In [1]:
from sys import path
path.append('../src/')

import cv2
import numpy as np
import pickle as pkl
from os.path import basename, splitext
from glob import glob
from utils import load
from mrk_file import MRKFile
from global_config import FOLDER_IMAGES, FOLDER_MRKS, FILE_DUMP_IMAGES, FILE_DUMP_MRKS, IMAGE_SIZE

In [2]:
print(FOLDER_IMAGES)
print(FOLDER_MRKS)
print(FILE_DUMP_IMAGES)
print(FILE_DUMP_MRKS)
print(IMAGE_SIZE)

../data/pybossa/images/
../data/pybossa/ground_truth/1_2_3_5_6_weight_2/
../data/pybossa/pybossa_images.pkl
../data/pybossa/pybossa_mrks.pkl
(224, 224)


# Data Loading 

In [3]:
image_files = np.array(glob(FOLDER_IMAGES + '*'))
mrk_files = np.array([f'{FOLDER_MRKS}{splitext(basename(path))[0]}.mrk' for path in image_files])

print(len(image_files))
print(len(mrk_files))
assert(len(image_files) == len(mrk_files))\

5763
5763


In [4]:
mrks = load.mrk_files_from_list_files(mrk_files)
print(mrks.shape, mrks.dtype)

pkl.dump((mrks, mrk_files), open(FILE_DUMP_MRKS, 'wb'), protocol=-1)

5763 of 5763
(5763,) object


In [5]:
x = load.images_from_list_files(image_files, output_size=IMAGE_SIZE, interpolation=cv2.INTER_AREA)
print(x.shape, x.dtype)

pkl.dump((x, image_files), open(FILE_DUMP_IMAGES, 'wb'), protocol=-1)

5763 of 5763
(5763, 224, 224, 3) float32
